In [16]:
# Importing recommended libraries
import os
from matplotlib import pyplot as plt
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG19

In [17]:
# Main directory where model data is saved
mainDataDirectory = "C:/Users/seyed/Music/Brain Tumor - Model Data/"

In [18]:
# Categories of image classification
imageCategories = ["Yes", "No"]

# Directories for training and testing data
trainDirectory = os.path.join(mainDataDirectory, 'Train')
testDirectory = os.path.join(mainDataDirectory, 'Test')

In [19]:
# Function to read images from a directory and add labels to each image
def read_images_from_directory(directory):
    images = []
    labels = []
    for category in imageCategories:
        category_path = os.path.join(directory, category)
        label = imageCategories.index(category)

        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)
            image = cv2.imread(image_path)
            images.append(image)
            if label == 0:
                labels.append("Yes")
            elif label == 1:
                labels.append("No")
    return images, labels

In [ ]:
# Read training images
train_x, train_y = read_images_from_directory(trainDirectory)

# Read testing images
test_x, test_y = read_images_from_directory(testDirectory)

# Convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [ ]:
# Checking for the number of data
train_x.shape

In [ ]:
test_x.shape

In [ ]:
# Checking the categories in the dataset
np.unique(train_y)

In [ ]:
# Map string labels to integer labels inorder for the model to read the data in integer format
label_mapping = {'No': 1, 'Yes': 0}

# Convert string labels to integer labels
train_y = [label_mapping[label] for label in train_y]
test_y = [label_mapping[label] for label in test_y]

# Convert labels to one-hot encoded format
train_y_one_hot = to_categorical(train_y, num_classes=2)
test_y_one_hot = to_categorical(test_y, num_classes=2)

In [ ]:
# Running the tf in cpu
with tf.device('/cpu:0'):
    # Loading the vgg16 model and setting up the image size to train
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    
    # Freeze the layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Build the model
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_x, train_y_one_hot, epochs=10, batch_size=16, validation_data=(test_x, test_y_one_hot))


In [ ]:
# Save the model
model.save('C:/Users/seyed/Music/Brain Tumor - Model/TumorDetectionModel_VGG-19.h5')

In [ ]:
model.summary()